# DSCI 100 Project Final Report

Miyuki, Jane, Rachel & Meg

### Introduction



We will be exploring video games and the people who play video games, specifically the game MineCraft. To do this, play sessions are being recorded as well as individual player information. This includes information such as a players demographic information, playing time, and their skill. From this, data sets are created and will be used for analysis and understanding.

To help determine “kinds” of players, our project aims to analyze the relationship between the age and the time played of players in the dataset.

The dataset that we will be using to determine the relationship between “age” and “time_played” is the “players.csv” data set. This data set contains information about the players experience, subscription status, hashedEmail, played_hours, name, gender and age. Since this data contains information on both the age and the played hours, this will be the most relevant in determining the relationship between the 2.

### Methods and Results

In [1]:
library(tidyverse)
library(dplyr)
library(ggplot2)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
players <- read_csv("players.csv") 
head(players)

ERROR: Error: 'players.csv' does not exist in current working directory ('/home/jovyan/work/plaicraft').


### Discussion

### References